# HW05: Credit Risk Prediction

Анализ и предсказание дефолта клиента по его финансовым характеристикам.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Загрузка и подготовка данных

In [ ]:
# Загружаем датасет
df = pd.read_csv('data.csv')
print(f'Shape: {df.shape}')
print(f'\nFirst rows:\n{df.head()}')
print(f'\nInfo:\n{df.info()}')
print(f'\nTarget distribution:\n{df["default"].value_counts()}')

In [ ]:
# Проверяем пропуски
print(f'Missing values:\n{df.isnull().sum().sum()}')

# Базовая статистика
print(f'\nDescribe:\n{df.describe()}')

## 2. EDA

In [ ]:
# Распределение целевой переменной
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df['default'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Default distribution')
axes[0].set_ylabel('Count')

df['default'].value_counts(normalize=True).plot(kind='pie', ax=axes[1], labels=['No Default', 'Default'], autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'])
axes[1].set_title('Default proportion')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Корреляция с таргетом
correlations = df.corr()['default'].sort_values(ascending=False)
print(f'Correlation with default:\n{correlations}')

In [ ]:
# Основные факторы
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Age vs default
df.boxplot(column='age', by='default', ax=axes[0, 0])
axes[0, 0].set_title('Age by Default')

# Income vs default
df.boxplot(column='income', by='default', ax=axes[0, 1])
axes[0, 1].set_title('Income by Default')

# Credit score vs default
df.boxplot(column='credit_score', by='default', ax=axes[1, 0])
axes[1, 0].set_title('Credit Score by Default')

# Debt to income vs default
df.boxplot(column='debt_to_income', by='default', ax=axes[1, 1])
axes[1, 1].set_title('Debt to Income by Default')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 3. Подготовка данных для моделирования

In [ ]:
# Разделяем признаки и таргет
X = df.drop(columns=['client_id', 'default'])
y = df['default']

print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')
print(f'\nFeatures: {list(X.columns)}')

In [ ]:
# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Train size: {X_train.shape}')
print(f'Test size: {X_test.shape}')
print(f'Train default rate: {y_train.mean():.3f}')
print(f'Test default rate: {y_test.mean():.3f}')

In [ ]:
# Масштабируем признаки
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Scaling completed')

## 4. Модели

In [ ]:
# Logistic Regression
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_scaled, y_train)

lr_pred = lr.predict(X_test_scaled)
lr_proba = lr.predict_proba(X_test_scaled)[:, 1]

print(f'LR ROC-AUC: {roc_auc_score(y_test, lr_proba):.4f}')
print(f'\nClassification Report (LR):\n{classification_report(y_test, lr_pred)}')

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)
rf_proba = rf.predict_proba(X_test)[:, 1]

print(f'RF ROC-AUC: {roc_auc_score(y_test, rf_proba):.4f}')
print(f'\nClassification Report (RF):\n{classification_report(y_test, rf_pred)}')

In [ ]:
# ROC curves
plt.figure(figsize=(10, 6))

fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_proba)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_proba)

plt.plot(fpr_lr, tpr_lr, label=f'LR (AUC={roc_auc_score(y_test, lr_proba):.4f})', linewidth=2)
plt.plot(fpr_rf, tpr_rf, label=f'RF (AUC={roc_auc_score(y_test, rf_proba):.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Feature importance
importances = rf.feature_importances_
feature_names = X.columns

feat_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feat_importance['feature'][:15], feat_importance['importance'][:15])
plt.xlabel('Importance')
plt.title('Top 15 Features (Random Forest)')
plt.tight_layout()
plt.show()

print(f'Top 10 features:\n{feat_importance.head(10)}')